In [92]:
from langchain_ollama import OllamaLLM
from langchain.chat_models import init_chat_model
import getpass
import os
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

llm = init_chat_model("llama3.1",model_provider = "Ollama")

In [93]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
loader = CSVLoader(
    file_path = 'nirf_clean.csv',
    source_column="Name",
    metadata_columns=["Rank","State"],
)
docs = loader.load()
embed = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": device}
    )
db = Chroma.from_documents(docs,embed,persist_directory="./chroma_db")
retiever = db.as_retriever(search_kargs={"k":3})

In [94]:

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = "Tavily-key"
from langchain_tavily import TavilySearch
search_tool = TavilySearch(
    max_results=5,
    topic="general"
    )

In [95]:
""" from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from accelerate import init_empty_weights
embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": device})
response = search_tool.invoke({"query": "What was the cutoff for nit trichy cs in 2024?"})
print(response)
raw_docs = response['results']
docs = [
    Document(page_content=res['content'],metadata={"url": res['url']}) for res in raw_docs
]
db = Chroma.from_documents(docs,embed,persist_directory="chroma_db")
"""

' from langchain_huggingface import HuggingFaceEmbeddings\nfrom langchain_core.documents import Document\nfrom accelerate import init_empty_weights\nembed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": device})\nresponse = search_tool.invoke({"query": "What was the cutoff for nit trichy cs in 2024?"})\nprint(response)\nraw_docs = response[\'results\']\ndocs = [\n    Document(page_content=res[\'content\'],metadata={"url": res[\'url\']}) for res in raw_docs\n]\ndb = Chroma.from_documents(docs,embed,persist_directory="chroma_db")\n'

In [96]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="history",
    input_key="input"
    )


In [97]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
template = """
You are an AI agent, specifically modeled to answer questions related to various engineering enterance exams in India.
You should ask the user various questions related to his exam scores, branch preferences, interests etc to access his profile, and then reccomend him 3 colleges
You must answer all the queires wihh the context provided below:
Here is some relevant context form old database : {context}
Here is some context taken from web: {web_context}
Here is the chat history: {history}
Here is the query : {input}
"""

prompt = ChatPromptTemplate.from_template(template)

document_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retiever,document_chain)


In [98]:
from langchain_core.documents import Document
from accelerate import init_empty_weights

print("Hello! I am a llm agent to help with all your doubts regarding the various diffrent engineering entrance exams in India." \
"Type 'exit' to quit")
while True:
    user_input = input("\nUser: ")
    if user_input == "exit":
        print("Exiting conversation.")
        break

    db_results = retiever.get_relevant_documents(user_input)
    context = "\n".join(res.page_content for res in db_results)

    web_response = search_tool.invoke({"query":user_input})
    raw = web_response['results']
    web_context = [
        Document(page_content=res['content'], metadata= {"url": res['url']}) for res in raw
    ]
    history_dict = memory.load_memory_variables({})
    history = history_dict.get("history", "")

    response = rag_chain.invoke({
        "input": user_input,
        "web_context": web_context,
        "context": context,
        "history": history,
    })
    memory.save_context({"input": user_input},{"output":response['answer']})
    print(f"AI: {response['answer']}")

Hello! I am a llm agent to help with all your doubts regarding the various diffrent engineering entrance exams in India.Type 'exit' to quit
AI: According to the latest updates from IIT Kanpur, the JEE Advanced 2025 exam date has been announced as May 18, 2025.

Now, let's move on to your profile. Can you please share your:

1. JEE Main score (out of 300) for this year?
2. Your preferred branch among Engineering, Architecture, or any other?
3. Any specific interests or hobbies that might help me suggest the best colleges for you?
AI: Congratulations on scoring a good JEE Main score!

To get personalized college recommendations, I need to know more about your profile. Please answer the following questions:

1. You mentioned that you scored **216** in JEE Mains. What percentile do you expect to achieve with this score? (You can refer to the web content provided for an idea of percentiles)
2. Which branch would you prefer among Civil, Mechanical, Electrical, Computer Science, or any other?

KeyError: 'results'